# Part III - Halo region

Now we will know how is the `Halo` region being considered for building expressions up. For that, we will use the time marching example shown in `01_iet` tutorial.

In [1]:
from devito import Eq, Grid, TimeFunction, Operator

grid = Grid(shape=(3, 3))
u = TimeFunction(name='u', grid=grid)
eq = Eq(u.forward, u+2)
op = Operator(eq)
print(u.data)

[[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]]


When capturing the constructed expression, we see $+ 1$ being added to the spatial indexes of $u$:

In [2]:
from devito.ir.iet import Expression, FindNodes
exprs = FindNodes(Expression).visit(op)
print(exprs[0].view)

<Expression u[t1, x + 1, y + 1] = u[t0, x + 1, y + 1] + 2>


That occurs because we have grid points surrounding the domain region, i.e. `ghost` points that are accessed by the stencil when iterating in the proximity of the domain boundary. This region is called `Halo`.

In [3]:
print(u.data_with_halo)

[[[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]]


Thus, the array accesses become logically aligned to the equation’s natural domain. For instance, given the usual Function $u(t, x)$ having one point on each side of the $x$ halo region, the array accesses $u[t, x]$ and $u[t, x + 2]$ are transformed, respectively, into $u[t, x + 1]$ and $u[t, x + 3]$. When $x = 0$, therefore, the values $u[t, 1]$ and $u[t, 3]$ are fetched, representing the first and third points in the computational domain.

By default, the Halo region have $1$ point on each side of the space dimensions. But this can be changed by passing a value in `space_order`:

In [4]:
u0 = TimeFunction(name='u0', grid=grid, space_order=0)
u2 = TimeFunction(name='u2', grid=grid, space_order=2)

In [5]:
u0.shape_with_halo

(2, 3, 3)

In [6]:
u2.shape_with_halo

(2, 7, 7)